In [1]:
#
# this code is copied from: https://github.com/mozilla/TTS/blob/master/notebooks/Benchmark.ipynb
#

# TTS
tts_pretrained_model = '/home/big-boy/Models/Blizzard/capacitron-April-06-2021_12+36PM-26e9ee0/best_model.pth.tar'
tts_pretrained_model_config = '/home/big-boy/projects/Capacitron/TTS/tts/configs/capacitron_blizzard.json'

import io
import torch 
import time
import numpy as np
from collections import OrderedDict
from matplotlib import pylab as plt
import IPython

%pylab inline
rcParams["figure.figsize"] = (16,5)

import librosa
import librosa.display

from TTS.tts.models.tacotron import Tacotron 
from TTS.tts.layers import *
from TTS.tts.utils.data import *
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config 
from TTS.tts.utils.generic_utils import setup_model

from TTS.tts.utils.text import text_to_sequence
from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.visual import visualize
from TTS.tts.utils.text.symbols import symbols, phonemes

from IPython.display import Audio

def tts(model, text, CONFIG, use_cuda, ap, use_gl, speaker_id=None, figures=True):
    t_1 = time.time()
    waveform, alignment, decoder_output, postnet_output, stop_tokens, inputs = synthesis(model, text, CONFIG, use_cuda, ap, truncated=True, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars)
    if CONFIG.model == "Tacotron" and not use_gl:
        postnet_output = ap.out_linear_to_mel(postnet_output.T).T
    if not use_gl:
        waveform = wavernn.generate(torch.FloatTensor(postnet_output.T).unsqueeze(0).cuda(), batched=batched_wavernn, target=11000, overlap=550)

    print(" >  Run-time: {}".format(time.time() - t_1))
    if figures:                                                                                                         
        visualize(alignment, postnet_output, stop_tokens, text, ap.hop_length, CONFIG, mel_spec)                                                                       
    # IPython.display.display(Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    # OUT_FOLDER = 'out/'
    # file_name = 'yolo.wav'
    # os.makedirs(OUT_FOLDER, exist_ok=True)
    # #file_name = text.replace(" ", "_").replace(".","") + ".wav"
    # out_path = os.path.join(OUT_FOLDER, file_name)
    # ap.save_wav(waveform, out_path)
    print(waveform)
    return alignment, postnet_output, stop_tokens, waveform
  
use_cuda = True
batched_wavernn = True

# initialize TTS
CONFIG = load_config(tts_pretrained_model_config)
# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, 1, CONFIG)
# load the audio processor
ap = AudioProcessor(**CONFIG.audio)         
# load model state
if use_cuda:
    cp = torch.load(tts_pretrained_model)
else:
    cp = torch.load(tts_pretrained_model, map_location=lambda storage, loc: storage)

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()
model.eval()
print(cp['step'])
model.decoder.max_decoder_steps = 2000

# # initialize WaveRNN
# VOCODER_CONFIG = load_config(wavernn_pretrained_model_config)
# with localimport('/content/WaveRNN') as _importer:
#   from models.wavernn import Model
# bits = 10

# wavernn = Model(
#         rnn_dims=512,
#         fc_dims=512,
#         mode="mold",
#         pad=2,
#         upsample_factors=VOCODER_CONFIG.upsample_factors,  # set this depending on dataset
#         feat_dims=VOCODER_CONFIG.audio["num_mels"],
#         compute_dims=128,
#         res_out_dims=128,
#         res_blocks=10,
#         hop_length=ap.hop_length,
#         sample_rate=ap.sample_rate,
#     ).cuda()
# check = torch.load(wavernn_pretrained_model)
# wavernn.load_state_dict(check['model'])
# if use_cuda:
#     wavernn.cuda()
# wavernn.eval()
# print(check['step'])

Populating the interactive namespace from numpy and matplotlib
/home/big-boy/anaconda3/envs/PyCapacitron/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +
 > Using model: Tacotron
 > Setting up Audio Processor...
 | > sample_rate:24000
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:95.0
 | > mel_fmax:12000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:None
 | > hop_length:256
 | > win_length:1024
15675


In [2]:
SENTENCE = 'Bill got in the habit of asking himself “Is that thought true?” And if he wasn’t absolutely certain it was, he just let it go.'
align, spec, stop_tokens, wav = tts(model, SENTENCE, CONFIG, use_cuda, ap, speaker_id=0, use_gl=True, figures=False)

RuntimeError: The expanded size of the tensor (119) must match the existing size (128) at non-singleton dimension 1.  Target sizes: [1, 119, -1].  Tensor sizes: [128, 1]